In [ ]:
!pip install -q --upgrade transformers==4.25.1 diffusers ftfy accelerate #there's probably more here if reqd for google colab but I'll work it out later

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.7 MB/s eta 0:00:00


In [5]:
from base64 import b64encode

import numpy
import torch
from diffusers import AutoencoderKL, LMSDiscreteScheduler, UNet2DConditionModel
from huggingface_hub import notebook_login

from PIL import Image
import requests
from io import BytesIO
import numpy as np


# For video display:
from IPython.display import HTML
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image
from torch import autocast
from torchvision import transforms as tfms
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer, logging
import os

import torch.nn.functional as F
torch.manual_seed(1)
#if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
if "mps" == torch_device: os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"

/mnt/c/Users/hew7/Documents/venvs/RF-demos/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Stable Diffusion explainer!

Hello! welcome to my explainer for stable diffusion - specifically targeting a non-technical audience. This will be a quick rundown of how Latent Diffusion Models work, specifically focusing on Stable Diffusion v1.4, as it's probably the most intuitive to explain. 

This explainer will be split into multiple parts - first, we will give a high level overview of the model cascade itself, then we'll go into each component of the cascade, and finally, we'll put the model cascade together at the end!

This is the first section out of 5: a high level explanation of the model cascade. The other sections are accessible here:

1. Intro to the architecture
2. CLIP Text embedder
3. Variational AutoEncoder
4. UNet denoiser
5. Conclusion - putting it all together

*turn these into links eventually*

## Part 2: text embedder

Let's start our deep dive first with the text embedder. As previously mentioned, the text embedder here is CLiP-ViT-L/14, which was fine-tuned specifically for text embedding and image embedding into the same vector space. Essentially, when you use this model to embed words and images, the words and images that are similar will give you very very similar numbers from the model. It's because this model was trained on both images and text that the folks at Stability.AI decided to use this model for their word embeddings!

Now let's load the model:

In [6]:
# Load the tokenizer and text encoder to tokenize and encode the text.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

text_encoder = text_encoder.to(torch_device).half()

To turn a sentence into an embedding, first we must "tokenize" the sentence, which is why we've loaded a Tokenizer.

A tokenizer takes subwords from your input string/sentence, and then, using a dictionary, converts them to integers. Subwords can be as short as a letter, or an entire word.

The reason we turn these into numbers first is to convert text into much smaller chunks of data that can more easily be mathematically operated on. Think of this stage as sort of like a simple code-book where we match certain chunks of words to numbers.

In [7]:
input_sentence=["Hi mum! I'm in a computer!"]

In [8]:
tokenized_input = tokenizer(input_sentence, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
#'padding' is to fill in "unused" tokens in the input, to make sure it goes into our embedding model properly
#'max_length' is setting how many tokens the padder needs to pad the input out to
#'truncation' being true means that, if we put in too many tokens, we will just chop off the last tokens to make sure the input fits
#'return_tensors' set to 'pt' means that our datatype will be compatible with PyTorch

print(tokenized_input)

{'input_ids': tensor([[49406,  1883,  4030,   256,   328,   880,   530,   320, 11639,   652,
           256, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])}


These tokenised outputs can easily be "reversed", to get our inputs back.

In [9]:
tokenizer.decode(tokenized_input['input_ids'][0], skip_special_tokens=True)

"hi mum! i'm in a computer!"

Now we will "embed" these tokens - or put them into a useful vector space for the model to "understand" what this sentence means.

We use the term "understand" loosely - really, this model is trained to keep points in vector embedding space close together if they're similar, and keep points far apart if they're not similar (like the dog and spaghetti stuff I was takling about earlier). What is and isn't similar is defined by humans who have labelled a bunch of data, as well as a bunch of other assumptions about language and context. We won't go too deep into it, but essentially, we've put this sentence into a vector space that we can now use to describe language in general.

In [10]:
tokenized_input = tokenized_input.to(torch_device)
sentence_embedding = text_encoder(**tokenized_input)

Now let's look at that shall we?

In [11]:
print(sentence_embedding)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.3884,  0.0229, -0.0522,  ..., -0.4902, -0.3066,  0.0674],
         [ 0.4297, -1.1533, -1.0186,  ..., -0.3411, -0.4497, -1.2920],
         [-1.0508,  0.8071, -1.4277,  ..., -0.5166,  1.0107,  0.9146],
         ...,
         [ 0.0632,  0.6182, -0.3821,  ..., -0.2637, -0.4199,  0.3838],
         [-0.0268,  0.4519, -0.2959,  ..., -0.3071, -0.3857,  0.2986],
         [-0.6709,  1.0703, -0.4839,  ...,  0.5630, -0.5439, -0.1895]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-4.5044e-01,  1.1367e+00, -7.3486e-01,  1.2295e+00,  1.1514e+00,
          9.9854e-01, -5.2832e-01, -9.7266e-01, -3.8989e-01, -6.5125e-02,
         -9.2285e-01,  1.5479e+00,  4.2786e-02, -8.6914e-01,  1.9014e+00,
         -1.7859e-01, -8.5938e-01,  1.8604e-01, -1.1641e+00,  6.5273e+00,
         -6.8994e-01,  9.7510e-01, -1.3975e+00,  2.6660e-01,  1.8005e-02,
         -6.5723e-01, -1.7609e-02, -8.4424e-01,

what you'll see printed out is two items - the top item is the "unpooled output" of the embedding model, and the bottom output is the "pooled" output. Essentially what's happening is that each token itself is getting an embedding assigned to it, and then all of those embeddings get averaged together in a sentence. This first sentence is the one we'll be putting into our UNet later.

Two things worth noting here - one is that it's now much trickier to decode this set of numbers (especially after they've been averaged) back into the input sentence. This is an encoder-only network, and typically the only way to turn this set of number back into a sentence is to use a decoder network trained to decode this embedding.

Two is that this is a set of numbers - and in that sense, you can sort of do an arbitrary amount of arithmetic on it to create whatever set of numbers you like! You could, for example, add 1 to all of the numbers in the vector, or multiply all the numbers, or add another vector, or completely randomise them! This is potentially an interesting way of exercising control over your final output in an LDM - however, we can start playing around with that later.

If you'd like - I've written a few fun functions that will do everything I just demonstrated, but with your own inputs. I'll use these later in this notebook, but feel free to play around with them for now.

In [12]:
def tokenize(stringlist):
    tokenized = tokenizer(stringlist, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    return tokenized

def detokenizer(tokenized):
    return tokenizer.decode(tokenized['input_ids'][0], skip_special_tokens=True)

def sentence_embedder(stringlist):
    inputs=tokenize(stringlist)
    inputs=inputs.to(torch_device)
    with torch.no_grad():
        sentence_embeddings=text_encoder(**inputs)
    return sentence_embeddings

In [13]:
demo_string=['demo string! The meaning of life is 42 (and so am I in spirit - 42 years old. Time flies.)']

tokenized=tokenize(demo_string)
decoded=detokenizer(tokenized)
embeddings=sentence_embedder(demo_string)

print(demo_string)
print('')
print(tokenized)
print('')
print(decoded)
print('')
print(embeddings)

['demo string! The meaning of life is 42 (and so am I in spirit - 42 years old. Time flies.)']

{'input_ids': tensor([[49406,  7380,  9696,   256,   518,  8342,   539,   970,   533,   275,
           273,   263,   537,   706,   687,   328,   530,  4071,   268,   275,
           273,  1151,   896,   269,   788,  8070,  5376, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 

In [14]:
# OPEN QUESTION: should I also embed some images and explain cos sim to show how CLIP works with images as well?

Now that we understand the text embedder, let's move on to the Variational AutoEncoder. *make this a link*